# LLaMA Training with PEFT

# Step 0: Install dependencies

In [1]:
%pip install pandas
%pip install datasets
%pip install seaborn
%pip install matplotlib
%pip install transformers
%pip install huggingface_hub
%pip install torch
%pip install peft
%pip install trl
%pip install tensorboard

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 2.2 MB/s eta 0:00:06
   - -------------------------------------- 0.3/11.6 MB 3.5 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.6 MB 5.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.7/11.6 MB 9.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.5/11.6 MB 10.4 MB/s eta 0:00:01
   ------------ --------------------------- 3.7/11.6 MB 13.1 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.6 MB 14.5 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.6 MB 15.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.2/11.6 MB 15.3 MB/s eta 0:00:01
   ------------------------- -------------- 7.5/11.6 MB 16.5 MB/s eta 0:00:01
   --------------------------- ------------ 8.1/11.6 MB 16.7 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.6 MB 16.2 MB/s eta 0:00:0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
weaviate-client 3.23.1 requires requests<=2.31.0,>=2.28.0, but you have requests 2.32.3 which is incompatible.
goldenverba 1.0.2 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.9.0-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.2.1-cp310-cp310-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/165.5 kB ? eta -:--:--
     ---------------- ---------------------- 71.7/165.5 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 165.5/165.5 kB 2.5 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp310-cp310-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.3.0-cp310-cp310-win_amd64.whl.metadata (9.4 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached matplotlib-3.9.0-cp310-cp310-win_amd64.whl (8.0 MB)
Using cached contourpy-1.2.1-cp310-cp310-win_amd64.whl (187 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ----------------------------------

### Inserting Your Hugging Face API Key

To access the gated model, you need to use your Hugging Face API key. You can insert your API key by running the following code:

In [2]:
# Insert your Hugging Face API key
token = "your_huggingface_api_key"
from huggingface_hub import notebook_login
notebook_login(token=token)


d:\verba_1.01\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: notebook_login() got an unexpected keyword argument 'token'

## Step 1: Data Preparation

In this step, we load the dataset, format the instructions and responses, and filter the examples based on token length.

In [4]:
# Import necessary libraries
import pandas as pd
from datasets import Dataset
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from huggingface_hub import login

# Replace 'your_huggingface_api_key' with your actual Hugging Face API key
api_key = 'your_huggingface_api_key'

# Log in to Hugging Face and save the token to git credentials helper
login(token=api_key, add_to_git_credential=True)

# Initialize tokenizer
base_model = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def prepare_data(csv_path):
    df = pd.read_csv(csv_path)

    df["formatted_instruction"] = df.apply(lambda x: f"### Instruction:\n{x['Instructions']}\n\n### Response:\n{x['Responses']}", axis=1)
    df["formatted_instruction_tok_len"] = df["formatted_instruction"].apply(lambda x: len(tokenizer(x)["input_ids"]))

    sns.boxplot(x=df["formatted_instruction_tok_len"])
    plt.xlabel("formatted_instruction_tok_len")
    plt.title("Token Length")
    plt.show()

    df = df[df["formatted_instruction_tok_len"] <= 128]
    dataset = Dataset.from_pandas(df)
    return dataset

# Specify the path to your dataset
csv_path = 'path_to_your_dataset.csv'
# Prepare the dataset
dataset = prepare_data(csv_path)


ValueError: Invalid token passed!

## Step 2: Model and Tokenizer Initialization

Here, we initialize the model and tokenizer. We use 4-bit quantization for efficient training.

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Set compute dtype
compute_dtype = torch.float16

# Configure quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Load the model with quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

## Step 3: Calculate Model Size

We define a function to calculate the model size in MB.

In [ ]:
# Function to calculate model size
def calculate_model_size(model):
    total_size = 0
    for param in model.parameters():
        param_size = param.numel() * param.element_size()
        total_size += param_size
    total_size_in_mb = total_size / (1024**2)
    return total_size_in_mb

# Calculate and print model size
model_size_mb = calculate_model_size(model)
print(f"Model size: {model_size_mb:.2f} MB")

## Step 4: PEFT Configuration

Set up PEFT parameters for training.

In [ ]:
# Import necessary libraries
from peft import LoraConfig

# Configure PEFT parameters
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

## Step 5: Training Arguments

Set the training arguments.

In [ ]:
# Import necessary libraries
from transformers import TrainingArguments

# Set training arguments
training_params = TrainingArguments(
    output_dir="./",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

## Step 6: SFTTrainer Setup and Training

Initialize `SFTTrainer` and train the model.

In [ ]:
# Import necessary libraries
from trl import SFTTrainer
import time

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="formatted_instruction",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

# Train the model
start = time.time()
output = trainer.train()
print("Time taken: ", time.time() - start)

## Step 7: Monitor Training with Tensorboard

Start Tensorboard for monitoring the training process.

In [ ]:
# Start Tensorboard for monitoring
from tensorboard import notebook

log_dir = "runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

## Step 8: Generate Text Using the Trained Model

Use the trained model to generate text based on a given prompt.

In [ ]:
# Import necessary libraries
from transformers import pipeline

# Initialize text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)

# Define prompt
prompt = "### Instruction:\nPlease generate 5 variants of the question: 'What is the capital of France?'\n\n### Response\n:"

# Generate text
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

### Implementation Notes

- Ensure you have the required libraries installed (`transformers`, `datasets`, `peft`, `trl`, `seaborn`, `matplotlib`, `tensorboard`).
- Adjust the file path for your dataset in `csv_path`.
- The token length filter and other parameters can be adjusted based on your specific use case and dataset.
- Monitor the training process using Tensorboard to check for a gradual decline in training loss.